In [ ]:
import numpy as np
import os
from src.nanobio_core.epic_cardio.measurement_load import load_measurement, wl_map_to_wells
from src.nanobio_core.epic_cardio.data_correction import correct_well
from src.nanobio_core.image_fitting.cardio_mic import CardioMicFitter, CardioMicScaling
from src.nanobio_core.image_fitting.single_cell_evaluator import CardioMicSingleCellEvaluator, SingleCellDisplayContour
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib
%matplotlib qt
import cv2

WIDTH = 80
WELL_IDS = [['C1', 'C2', 'C3', 'C4'], ['B1', 'B2', 'B3', 'B4'], ['A1', 'A2', 'A3', 'A4']] # Tükrözés korrigálva
def flatten(t):
    return [item for sublist in t for item in sublist]
WELL_NAMES = flatten(WELL_IDS)

# Well mappa elérési útvonala
CARDIO_PATH = '../../data/classification_datasets/fibronectin/20210527_HepG2_fn/epic_data/'
MIC_PATH = '../../data/classification_datasets/lognorm-images/fibronectin/annotated/20210527_HepG2_fn/'
PRED_PATH = '/home/balint/projects/lognorm-images/fibronectin/20210527_HepG2_fn/'

RESULT_PATH = os.path.join(CARDIO_PATH, 'result')

if not os.path.exists(RESULT_PATH):
    os.mkdir(RESULT_PATH)

In [ ]:
# Betölti a 3x4-es well képet a projekt mappából.
WL_map, time = load_measurement(CARDIO_PATH)
# Itt szétválasztásra kerülnek a wellek. Betöltéskor egy 240x320-as képen található a 3x4 elrendezésű 12 well.
wells = wl_map_to_wells(WL_map, flip=True)
phases = list(np.where((np.diff(time)) > 60)[0] + 1)
[(n+1, p) for n, p in enumerate(phases)]

In [ ]:
# Sejt szűrés a wellekből.
well_data = {}
cut_point = 1
signal_start = 0 if cut_point == 0 or cut_point > len(phases) else phases[cut_point - 1]
signal_start += 5
time = time[signal_start:]
time -= time[0]

for name in WELL_NAMES:
    print("Parsing", name, end='\r')
    well_tmp = wells[name]
    well_corr = well_tmp[signal_start:]
    well_corr = correct_well(well_corr)
    well_data[name] = well_corr
print("Parsing finished!")

In [ ]:
# for well_id in WELL_NAMES:
#     mic_path = f'{MIC_PATH}/{well_id}.jpeg'
#     annot_path = f'{MIC_PATH}/{well_id}_seg.npy'
#     params_path = f'{CARDIO_PATH}/result/metadata/{well_id}_map_params.json'
    
#     if not os.path.exists(params_path):
#         continue

#     img = cv2.imread(mic_path)
#     well = well_data[well_id]

#     im_src = well
#     dat = np.load(annot_path, allow_pickle=True).item()
#     mask = dat['masks'][0]

#     filter_params = {
#         'area': (0, np.Inf),
#         'max_value': (100, np.Inf),
#         'adjacent': False,
#     }

#     evaluator = CardioMicSingleCellEvaluator(time, well, img, mask, params_path, load_selection=None, filter_params=filter_params)

#     evaluator.select_all()

#     evaluator.save(RESULT_PATH, well_id, 4)

In [58]:
well_id = 'A1'
mic_path = f'{MIC_PATH}/{well_id}.jpeg'
annot_path = f'{MIC_PATH}/{well_id}_seg.npy'
pred_path = f'{PRED_PATH}/{well_id}_pred_seg.npy'
params_path = f'{CARDIO_PATH}/result/metadata/{well_id}_map_params.json'

img = cv2.imread(mic_path)
well = well_data[well_id]

im_src = well
mask = np.load(annot_path, allow_pickle=True).item()['masks'][0]
mask_pred = np.load(pred_path, allow_pickle=True).item()['masks-tuned']

filter_params = {
    'area': (0, np.Inf),
    'max_value': (75, np.Inf),
    'adjacent': False,
}

evaluator = CardioMicSingleCellEvaluator(time, well, img, mask, mask_pred, params_path, 
                                         load_selection=None, filter_params=filter_params, ws_threshold=50)
print('select')
evaluator.select_all()
print('save')
evaluator.save(RESULT_PATH, well_id, 4)

select
save
Duration 0:00:24.743822


In [ ]:
RESOLUTION = 0.5
evaluator.display(RESOLUTION, 3, display_contours=[SingleCellDisplayContour.ALL])

In [ ]:
evaluator.save(RESULT_PATH, well_id, 4)

In [59]:
seg = np.load(f'{RESULT_PATH}/{well_id}_seg.npz')
im_cardio, im_watershed, im_cover, im_pred = seg['cardio'], seg['cardio_watershed'], seg['cardio_cover'], seg['cardio_pred']
idx = 0

In [82]:
fig, ax = plt.subplots(2, 2)
ax[0, 0].imshow(im_cardio[idx, -1], vmin=0, vmax=np.max(im_cardio))
ax[1, 0].imshow(im_pred[idx, -1], vmin=0, vmax=np.max(im_cardio))
ax[0, 1].imshow(im_watershed[idx, -1], vmin=0, vmax=np.max(im_cardio))
ax[1, 1].imshow(im_cover[idx, -1], vmin=0, vmax=np.max(im_cardio))
plt.show()
idx+=1

In [60]:
im_cardio.shape, im_cover.shape, im_pred.shape, im_watershed.shape

((58, 772, 8, 8), (58, 772, 8, 8), (55, 772, 8, 8), (54, 772, 8, 8))